In [259]:
import psycopg2
import sys
import pandas as pd
import numpy as np

conn_string="host='localhost' dbname='BBS' user='postgres' password='Azurarai1994'"
conn=psycopg2.connect(conn_string)
cursor=conn.cursor()

In [260]:
def get_data(ESIID):
    #Run SQL query
    command=""" SELECT t3."LSEData", t3."LseHeaderID"
    FROM public."LSEInfos" AS t3
    WHERE t3."LseHeaderID" IN ( SELECT t2."LseHeaderID" FROM public."LSEHeader" AS t2 WHERE t2."ESIID" = '%s')""" %(ESIID )
#    cursor.execute("""
#    SELECT t3."LSEData", t3."LseHeaderID"
#    FROM public."LSEInfos" AS t3
#    WHERE t3."LseHeaderID" IN ( SELECT t2."LseHeaderID" FROM public."LSEHeader" AS t2 WHERE t2."ESIID"=ESIID)
#    """)
    cursor.execute(command)
    conn.commit()

    #get record

    rec_consumption=cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]

    # convert to dataframe

    df1=pd.DataFrame(rec_consumption,columns=colnames)
    return df1

In [261]:
def get_info(ESIID):
    command1="""SELECT t1."TDSPName",t1."ServiceAddress",t1."ServiceZip",t2."LseHeaderID",t2."StartTime",t2."StopTime"
    FROM public."CustomerRawData" AS t1,public."LSEHeader" AS t2
    WHERE t1."PremiseID" = '%s'"""%(ESIID)
    command2="""AND t2."ESIID"='%s'"""%(ESIID)
    command=command1+command2
    cursor.execute(command)
    conn.commit()

    #get record

    rec_info=cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]

    # convert to dataframe

    df2=pd.DataFrame(rec_info,columns=colnames)
    return df2

def annual_cons(df2,new_df):
    annual_cons_rec=pd.merge(df2,new_df,left_on=df2.columns[2],right_on=new_df.columns[0])
    return annual_cons_rec

In [262]:
def get_hourly_consumption(df1):
    s=pd.Series(df1['LSEData'])
    s=s.str.split(",([A-Z]),,",n=4,expand=True)

    df1['1']=s[:][0].astype('float')
    df1['2']=s[:][2].astype('float')
    df1['3']=s[:][4].astype('float')
    df1['4']=s[:][6].astype('float')
    df1['hourly_consumption']=df1['1']+df1['2']+df1['3']+df1['4']
    return df1

In [1]:
def get_daily_consumption(df1):
    head_id=df1['LseHeaderID'].unique()
    new_df = pd.DataFrame(columns=['Lse_header','daily_consumption'])

    daily_consumption=np.zeros(len(head_id))
    header_id=[]
    count=0
    for i in head_id:
        temp_df=df1[df1.LseHeaderID==i]
        daily_consumption[count]=temp_df['hourly_consumption'].sum()
        header_id.append(i)
        count=count+1

    new_df.Lse_header=head_id
    new_df.daily_consumption=daily_consumption
    new_data=new_data.sort_values(by='StartTime')
    return new_df

In [264]:
cursor.execute("""
SELECT t1.acc,t1.bld_Sty_code,t1.depreciation_val,t1.qlty_Des,t1.date_erected,t1.gross_area,t2."PremiseID",t2."ServiceAddress",t2."ServiceCity",t2."ServiceZip"
FROM building_res AS t1, res_match AS t2
WHERE t1.acc= t2.acc
LIMIT 200
""")
conn.commit()

#get record

rec_customer=cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]

# convert to dataframe

df_customer=pd.DataFrame(rec_customer,columns=colnames)
df_customer.head(5)

,acc,bld_sty_code,depreciation_val,qlty_des,date_erected,gross_area,PremiseID,ServiceAddress,ServiceCity,ServiceZip
0,1115070000015,101,188077,Good,1982,3478,1008901020194175387100,4410 MONTEVIDEO CIR,PASADENA,77504
1,1096230000026,101,118052,Average,1977,2210,1008901019548447245100,23939 BEAVERWOOD DR,SPRING,77373
2,1122030000003,101,199472,Good,1980,5164,1008901033585434503100,3 LIVE OAK TRL,Cypress,77429
3,1095310000011,101,188864,Good,1979,3448,1008901001155652900100,18623 CANDLEVIEW DR,SPRING,77388
4,1096560000052,101,137124,Average,1976,2935,1008901020147797019100,10534 KIRKWREN DR,HOUSTON,77089


In [266]:
new_df=[]
cons=np.zeros(200)
for k in range(0,200):
    ESIID=df_customer.PremiseID[k]
    df1=get_data(ESIID)
    df_h=get_hourly_consumption(df1)
    df_d=get_daily_consumption(df_h)
    df2=get_info(ESIID)
    #annual_cons_rec=annual_cons(df2,df_d)
    #annual_hourly_cons_rec=pd.merge(df2,df_h,left_on=df2.columns[2],right_on=df_h.columns[1])
    #file_h="C:/Users/ruchi/Desktop/Result1/C100_RES/hourly_consumption"+ESIID+".txt"
    #annual_hourly_cons_rec.to_dense().to_csv(file_h, index = False, sep=',', encoding='utf-8')
    #file_d="C:/Users/ruchi/Desktop/Result1/C100_RES/daily_consumption"+ESIID+".txt"
    #annual_cons_rec.to_dense().to_csv(file_d, index = False, sep=',', encoding='utf-8')
    cons[k]=sum(df_d.daily_consumption)

new_df = pd.DataFrame(columns=['ESIID','annual_consumption'])
new_df.ESIID=df_customer.PremiseID
new_df.annual_consumption=cons
annual_cons_data=pd.merge(df_customer,new_df,left_on=df_customer.columns[6],right_on=new_df.columns[0])
file_a="C:/Users/ruchi/Desktop/annual_consumption_100.csv"
annual_cons_data.to_dense().to_csv(file_a, index = False, sep=',', encoding='utf-8') 

In [204]:
cursor.execute("""
SELECT *
FROM res_match
LIMIT 100
""")
conn.commit()

#get record

rec_bld=cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]

# convert to dataframe

df_bld=pd.DataFrame(rec_bld,columns=colnames)
df_bld.head(5)


,acc,use_code,bld_no,imprv_type,bld_sty_code,class_struct,class_struct_des,depreciation_val,cama_replacement_cost,accrued_depr_pct,...,nbhd_factor,rcnld,size_index,lump_sum_adj,PremiseID,PlanGroup,ServiceAddress,ServiceCity,ServiceZip,acc
0,1000740000005,A1,1,1001,101,R,Residential,190566,394711,0.68,...,1.48,128760.81,0.8,27698,1008901010187475575100,Residential,14934 BROADGREEN DR,HOUSTON,77079,1000740000005
1,1014010000010,A1,1,1001,101,R,Residential,156986,253203,0.62,...,1.8,87214.44,0.87,15781,1008901012189203332100,Residential,12850 WESTLEIGH DR,HOUSTON,77077,1014010000010
2,1022570000035,A1,1,1001,101,R,Residential,186933,333809,0.56,...,1.53,122178.43,0.79,27018,1008901040561327541100,Residential,6111 ELMGROVE RD,SPRING,77389,1022570000035
3,1022580000040,A1,1,1001,101,R,Residential,180622,328403,0.55,...,1.53,118053.59,0.78,19215,1008901039561000170100,Residential,6410 ELMGROVE RD,SPRING,77389,1022580000040
4,1001000000016,A1,1,1001,101,R,Residential,149804,288085,0.52,...,1.89,79261.38,0.84,15343,1008901018631687105100,Residential,15419 DIANA LN,HOUSTON,77062,1001000000016


In [209]:
annual_cons_data_new=pd.merge(new_df,df_bld,left_on=new_df.columns[0],right_on=df_bld.columns[30])
file_a="C:/Users/ruchi/Desktop/annual_consumption_100_merged.csv"
annual_cons_data_new.to_dense().to_csv(file_a, index = False, sep=',', encoding='utf-8') 

In [268]:
new_df.head(5)

,ESIID,annual_consumption
0,1008901020194175387100,16123.649
1,1008901019548447245100,14695.383
2,1008901033585434503100,29432.657
3,1008901001155652900100,17235.664
4,1008901020147797019100,11880.737


In [270]:
annual_cons_data=pd.merge(df_customer,new_df,left_on=df_customer.columns[6],right_on=new_df.columns[0])


In [269]:
df_customer.head(5)

,acc,bld_sty_code,depreciation_val,qlty_des,date_erected,gross_area,PremiseID,ServiceAddress,ServiceCity,ServiceZip
0,1115070000015,101,188077,Good,1982,3478,1008901020194175387100,4410 MONTEVIDEO CIR,PASADENA,77504
1,1096230000026,101,118052,Average,1977,2210,1008901019548447245100,23939 BEAVERWOOD DR,SPRING,77373
2,1122030000003,101,199472,Good,1980,5164,1008901033585434503100,3 LIVE OAK TRL,Cypress,77429
3,1095310000011,101,188864,Good,1979,3448,1008901001155652900100,18623 CANDLEVIEW DR,SPRING,77388
4,1096560000052,101,137124,Average,1976,2935,1008901020147797019100,10534 KIRKWREN DR,HOUSTON,77089


In [271]:
annual_cons_data.head(5)

,acc,bld_sty_code,depreciation_val,qlty_des,date_erected,gross_area,PremiseID,ServiceAddress,ServiceCity,ServiceZip,ESIID,annual_consumption
0,1115070000015,101,188077,Good,1982,3478,1008901020194175387100,4410 MONTEVIDEO CIR,PASADENA,77504,1008901020194175387100,16123.649
1,1096230000026,101,118052,Average,1977,2210,1008901019548447245100,23939 BEAVERWOOD DR,SPRING,77373,1008901019548447245100,14695.383
2,1122030000003,101,199472,Good,1980,5164,1008901033585434503100,3 LIVE OAK TRL,Cypress,77429,1008901033585434503100,29432.657
3,1095310000011,101,188864,Good,1979,3448,1008901001155652900100,18623 CANDLEVIEW DR,SPRING,77388,1008901001155652900100,17235.664
4,1096560000052,101,137124,Average,1976,2935,1008901020147797019100,10534 KIRKWREN DR,HOUSTON,77089,1008901020147797019100,11880.737


In [273]:
file_a="C:/Users/ruchi/Desktop/annual_consumption_100_res.csv"
annual_cons_data.to_dense().to_csv(file_a, index = False, sep=',', encoding='utf-8') 